In [8]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse
from cellpose import core, utils, io, models, metrics
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
logger_setup();

2022-10-17 10:34:31,670 [INFO] ** TORCH CUDA version installed and working. **
>>> GPU activated? YES
2022-10-17 10:34:31,671 [INFO] WRITING LOG OUTPUT TO /home/pablo/.cellpose/run.log


# Transform 3D data to 2D for users to correct

## If required, run prediction on new images

In [ ]:
from cellpose import models

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')## or T2

# Train a new model with your data

In [ ]:
# DEFINE CELLPOSE MODEL (without size model)
model = models.CellposeModel(gpu=use_GPU, model_type=initial_model)

# set channels
channels = [chan, chan2]

# get files
output = io.load_train_test_data(train_dir, test_dir, mask_filter='_seg.npy')
train_data, train_labels, _, test_data, test_labels, _ = output

new_model_path = model.train(train_data, train_labels, 
                              test_data=test_data,
                              test_labels=test_labels,
                              channels=channels, 
                              save_path=train_dir, 
                              n_epochs=n_epochs,
                              learning_rate=learning_rate, 
                              weight_decay=weight_decay, 
                              nimg_per_epoch=8,
                              model_name=model_name)

# diameter of labels in training images
diam_labels = model.diam_labels.copy()

# Using the new model created to predict 3D data

In [13]:
files_path = '/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc'
model_path = files_path + '/models/'

In [14]:
# gets image files in dir (ignoring image files ending in _masks)
files = io.get_image_files(files_path, '_masks')
print(files)
images = [io.imread(f) for f in files]

# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# use model diameter if user diameter is 0
diameter = model.diam_labels if diameter==0 else diameter

# * with 3D you have to set the diameter manually (no auto detect) *

### TWO WAYS TO RUN CELLPOSE IN 3D

# 1. computes flows from 2D slices and combines into 3D flows to create masks
print('running cellpose 2D slice flows => masks')
masks, flows, styles, _ = model.eval(images, diameter=30, do_3D=True, 
                                     channels=[chan, chan2], 
                                     flow_threshold=flow_threshold,
                                     cellprob_threshold=cellprob_threshold)

# 2. computes masks in 2D slices and stitches masks in 3D based on mask overlap
print('running cellpose 2D + stitching masks')
masks_stitched, flows_stitched, styles_stitched, _ = model.eval(images, channels=[chan, chan2], diameter=diameter, do_3D=False, stitch_threshold=0.5)

['/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc/210520_NubG4-UASmyrGFP_CELL-TAK-FLAT_DISH-1-DISC-3_STACK_MEDIA0000.tif', '/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc/210520_NubG4-UASmyrGFP_CELL-TAK-FLAT_DISH-1-DISC-3_STACK_MEDIA0001.tif', '/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc/210520_NubG4-UASmyrGFP_CELL-TAK-FLAT_DISH-1-DISC-3_STACK_MEDIA0002.tif', '/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc/210520_NubG4-UASmyrGFP_CELL-TAK-FLAT_DISH-1-DISC-3_STACK_MEDIA0003.tif', '/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc/210520_NubG4-UASmyrGFP_CELL-TAK-FLAT_DISH-1-DISC-3_STACK_MEDIA0004.tif', '/media/pablo/

2022-10-17 11:07:08,567 [INFO] >>>> loading model /media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc/models/
2022-10-17 11:07:08,571 [INFO] ** TORCH CUDA version installed and working. **
2022-10-17 11:07:08,572 [INFO] >>>> using GPU


IsADirectoryError: [Errno 21] Is a directory: '/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc/models/'

In [ ]:
# DISPLAY RESULTS 3D flows => masks
plt.figure(figsize=(15,3))
for i,iplane in enumerate(np.arange(0,75,10,int)):
  img0 = plot.image_to_rgb(img_3D[iplane, [1,0]].copy(), channels=[2,3])
  plt.subplot(1,8,i+1)
  outlines = utils.masks_to_outlines(masks[iplane])
  outX, outY = np.nonzero(outlines)
  imgout= img0.copy()
  imgout[outX, outY] = np.array([255,75,75])
  plt.imshow(imgout)
  plt.title('iplane = %d'%iplane)

In [ ]:
# DISPLAY RESULTS stitching
plt.figure(figsize=(15,3))
for i,iplane in enumerate(np.arange(0,75,10,int)):
  img0 = plot.image_to_rgb(img_3D[iplane, [1,0]].copy(), channels=[2,3])
  plt.subplot(1,8,i+1)
  outlines = utils.masks_to_outlines(masks_stitched[iplane])
  outX, outY = np.nonzero(outlines)
  imgout= img0.copy()
  imgout[outX, outY] = np.array([255,75,75])
  plt.imshow(imgout)
  plt.title('iplane = %d'%iplane)

In [ ]:
from cellpose import io

io.masks_flows_to_seg(images, 
                      masks, 
                      flows, 
                      diameter*np.ones(len(masks)), 
                      files, 
                      channels)